In [1]:
import sys

# This to get the peerless target star DataFrame for example purposes
sys.path.append('/u/tdm/repositories/peerless/prediction')
sys.path.append('/u/tdm/repositories/peerless')
from targets import targets

# The action is here. Depends on vespa & isochrones.
from exosyspop.populations import KeplerBinaryPopulation

/u/tdm/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [2]:
pop = KeplerBinaryPopulation(targets, fB=0.4)

In [3]:
# Accessing secondary properties will initialize a secondary simulation,
# calling pop._generate_binaries().  The first time this is called, the
# secondary property regressors get trained.
pop.radius_B

dmag regressor trained, R2=0.999277123516
q regressor trained, R2=0.999638706877


NameError: global name 'M2' is not defined

In [4]:
# subsequent calls are much faster; e.g.
pop._generate_binaries()
print(pop.radius_B)
%timeit pop._generate_binaries()

[        nan  0.6226095   1.28837028 ...,  0.32688018  0.76188309
         nan]
10 loops, best of 3: 69 ms per loop


In [5]:
# Similarly, accessing orbital properties will generate them
pop.period

array([  2.39923898e+07,   3.02891869e+02,   2.84246930e+02, ...,
         8.93145353e+01,   5.73817046e+02,   8.08750435e+06])

# Synthetic observations

In [6]:
# Now, we can observe and see what we see.  This takes into account
# duty cycle & data span, as well as geometry.
obs = pop.observe()
print(len(obs))
print(obs.columns)
obs.head()

472
Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec'],
      dtype='object')


,index,period,ecc,w,inc,a,aR,b_pri,b_sec,k,...,d_sec,T14_pri,T14_sec,T23_pri,T23_sec,dataspan,dutycycle,flux_ratio,n_pri,n_sec
0,447,14.602402,0.070548,2.205994,1.529148,2.044720e+12,31.989808,1.254088,1.405098,0.483727,...,0.000621,0.108852,0.073302,0,0,1459.789,0.6988,0.027516,71,77
1,495,1.242214,0.821637,5.760657,1.056559,4.187386e+11,6.546401,1.773413,0.741965,0.708778,...,0.090335,0.000000,0.043710,0,0,1459.789,0.8751,0.155126,0,1030
2,826,4.130219,0.636983,5.104982,1.419239,8.739236e+11,17.190426,3.748217,0.970910,0.763246,...,0.062912,0.000000,0.055328,0,0,1459.789,0.8751,0.163678,0,301
3,828,1.360512,0.673847,0.964521,1.132260,4.285606e+11,6.688196,0.997837,3.474276,0.610414,...,0.000000,0.043308,0.000000,0,0,1459.789,0.8750,0.062644,949,0
4,874,9.915634,0.000000,3.609366,1.536560,1.724785e+12,24.055844,0.823419,0.823419,0.337890,...,0.004730,0.138479,0.138479,0,0,1459.789,0.8752,0.005849,125,139


In [7]:
# This is pretty fast, even when generating a new population each time:
%timeit pop.observe(new=True)

10 loops, best of 3: 150 ms per loop


In [8]:
# Even faster if we only generate new orbits.
%timeit pop.observe(new_orbits=True)

10 loops, best of 3: 82.1 ms per loop


In [9]:
# So we can predict the expected number of observations pretty easily.
import numpy as np
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True)) for i in range(N)])
n_obs.mean(), n_obs.std()

(507.82999999999998, 21.487231091976462)

In [10]:
# Notice that the above does not yet have trapezoidal parameters.  There are two options to generate these.
# Either we can set the fit_trap parameter, as follows:
obs = pop.observe(fit_trap=True)
print(len(obs))
obs.columns

495


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'trap_dur_pri', u'trap_depth_pri',
       u'trap_slope_pri', u'trap_dur_sec', u'trap_depth_sec',
       u'trap_slope_sec'],
      dtype='object')

In [11]:
# All things considered, this is still pretty fast if we just need to do it a few times:
%timeit pop.observe(fit_trap=True)

1 loops, best of 3: 2.32 s per loop


In [12]:
# However, this is pretty slow if we want to do inference.  To help with this, we can 
# tell it to train & use a regression.  Training only happens once; by default with 10,000 
# synthetic observations.  This takes a minute or so.
obs = pop.observe(regr_trap=True)
print(len(obs))
obs.columns

Depth trained: R2=0.99762093605
Duration trained: R2=0.938498403019
Slope trained: R2=0.92227570434
495


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [13]:
# Subsequent calls are much faster
%timeit pop.observe(regr_trap=True)

10 loops, best of 3: 31.9 ms per loop


In [14]:
# Even generating a new stellar population & observing it is pretty quick
%timeit pop.observe(regr_trap=True, new=True)

10 loops, best of 3: 173 ms per loop


In [15]:
# Or again, you can just generate new orbits (rather than new binaries & new orbits)
%timeit pop.observe(regr_trap=True, new_orbits=True)

10 loops, best of 3: 102 ms per loop


In [16]:
# Generating the training data used for the trapezoid shape regression above used
# this function, which can be otherwise useful to sample >N random observations 
# from the existing population.  `trap_regr` defaults to `True` here.  
# This function also takes `new` or `new_orbits` keywords.
obs_pop = pop.get_N_observed(N=10000, new_orbits=True)
print(len(obs_pop))
obs_pop.columns

10121


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [17]:
# We can now look, e.g. at the expected number of single/double eclipsing systems:
query = '(n_pri < 3) & (n_sec < 3) & (n_pri==0 | n_sec==0)'
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(9.4299999999999997, 2.7722012913928165)